In [3]:
from typing import Dict, List
import requests
#headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AurlleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

categories = {
    'all' : 2,
    'tops' : 30,
    'jackets': 31,
    'pants': 32,
    'shoes': 33,
    'bag': 34,
    'suit': 35,
    'hats': 36,
    'rings': 37,
    'glasses': 38,
    'watches': 39,
    'socks/leggings': 40,
    'boxers': 41,
    'misc': 42
}

brands = {
    'number nine':873,
    'bape': 136
}

sizes_clothing = {
    'S':2,
    'M':3,
    'L':4,
    'XL':5
}

sizes_shoes = {
    '9.5':136,
    '10':137,
    '10.5':138,
    '11': 139,
    '11.5':140,
    '12':141,
    'biggg': 143
}

def gen_brands(brand:List[str]):
    return '%2C'.join(map(lambda x:str(brands.get(x)),brand))

def gen_sizes(sizes:List[str],sz: Dict[str,int]):
    return '%2C'.join(map(lambda x:str(sz.get(x)),sizes))

def build_url(keywords:str,category:str,brand:List[str],clothing_size:bool,sizes:List[str],new_order:bool):
    '''A function that generates a Mercari JP search url from parameters'''
    url = 'http://jp.mercari.com/search?'
    kwrds = f'keyword={"%20".join(keywords.split())}'
    order='&sort=created_time&order=desc'
    url += kwrds
    if new_order:
        url += order
    url += f'&category_id={categories[category]}'
    url += f'&brand_id={gen_brands(brand)}'
    url += f'&status=on_sale'
    if clothing_size:
        url += f'&size_id={gen_sizes(sizes,sizes_clothing)}'
    else:
        url += f'&size_id={gen_sizes(sizes,sizes_shoes)}'
    return url

build_url('gotham','tops',['number nine'],True,['L','XL'],True)





'http://jp.mercari.com/search?keyword=gotham&sort=created_time&order=desc&category_id=30&brand_id=873&status=on_sale&size_id=4%2C5'

In [50]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.content,'html.parser')

In [106]:
from lib2to3.pgen2.token import OP
from time import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from webdriver_manager.firefox import GeckoDriverManager

import time

options = Options()
options.add_argument('--disable-gpu') 

driver = webdriver.Firefox(options=options)


In [107]:
def selenium_fetch(driver:webdriver.Firefox,url:str):
    driver.get(url)
    try:
        elem = WebDriverWait(driver,4).until(EC.presence_of_element_located((By.CLASS_NAME,'mer-item-thumbnail')))
    except TimeoutException:
        print('loading took 2 long')

selenium_fetch(driver,pp)
soup = BeautifulSoup(driver.page_source,'html.parser')
soup.find_all(attrs={'id':'search-result'}) 


TypeError: Object of type function is not JSON serializable

In [108]:
items = soup.find_all('mer-item-thumbnail')
first_forward = '/html/body/div[1]/div[1]/div/div/div/main/div/div[2]/div/div/div/div[1]/mer-button/button'
second_forward = '/html/body/div/div[1]/div/div/div/main/div/div[2]/div/div/div/div[1]/mer-button[2]/button'
elem = None
while(soup.find(attrs={'data-testid':'pagination-next-button'})!=None):
    elem = driver.find_element(By.XPATH,first_forward)
    elem.click()
    WebDriverWait(driver,4)
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    items += soup.find_all('mer-item-thumbnail')

NoSuchElementException: Message: Unable to locate element: mer-button:nth-child(3) > button:nth-child(1)
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.jsm:12:1
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:404:5
element.find/</<@chrome://remote/content/marionette/element.js:291:16


In [100]:
driver.close()

In [80]:
items[0].attrs



    


IndexError: list index out of range

In [79]:
from typing import List
from pydantic import BaseModel

class MercariItem(BaseModel):
    name : str
    src : str
    item_id : str
    price : str


def get_item_info(items:List[str]) -> List[tuple]:
    ret = []
    for item in items:
        attrs = item.attrs
        name = attrs['item-name']
        src = attrs['src']
        item_id = src.split('/')[-1].split('_')[0]
        price = attrs['price']
        tmp = MercariItem(name=name,src=src,item_id=item_id,price=price)
        ret.append(tmp)
        return ret
get_item_info(items)


In [4]:
items[0]

NameError: name 'items' is not defined